In [1]:
import ee
import geemap

In [29]:
canales_roi=ee.Geometry.Polygon([[-3.481207, 37.148272],
      [-3.481207, 37.163187],
      [-3.43606, 37.163187],
      [-3.43606, 37.148272],
      [-3.481207, 37.148272]])
quentar_roi=ee.Geometry.Polygon([[-3.444471, 37.199743],
      [-3.444471, 37.212048],
      [-3.421512, 37.212048],
      [-3.421512, 37.199743],
      [-3.444471, 37.199743]])

colomera_roi=ee.Geometry.Polygon([[-3.738956, 37.39696],
      [-3.738956, 37.426343],
      [-3.684111, 37.426343],
      [-3.684111, 37.39696],
      [-3.738956, 37.39696]])

iznajar_roi=ee.Geometry.Polygon([[-4.390068, 37.233574],
      [-4.390068, 37.307632],
      [-4.213943, 37.307632],
      [-4.213943, 37.233574],
      [-4.390068, 37.233574]])
bermejales_roi=ee.Geometry.Polygon([[-3.916969, 36.949292],
      [-3.916969, 37.007025],
      [-3.853283, 37.007025],
      [-3.853283, 36.949292],
      [-3.916969, 36.949292]])
cubillas_roi=ee.Geometry.Polygon([[-3.691578, 37.265609],
      [-3.691578, 37.285006],
      [-3.655186, 37.285006],
      [-3.655186, 37.265609],
      [-3.691578, 37.265609]])
elgergal_roi=ee.Geometry.Polygon([[-6.061192, 37.563375],
      [-6.061192, 37.605137],
      [-6.028919, 37.605137],
      [-6.028919, 37.563375],
      [-6.061192, 37.563375]])

lakes_names = {
    'canales_boundary': canales_roi,
    'quentar_boundary': quentar_roi,
    'colomera_boundary': colomera_roi,
    'iznajar_boundary': iznajar_roi,
    'bermejales_boundary': bermejales_roi,
    'cubillas_boundary': cubillas_roi,
    'elgergal_boundary': elgergal_roi
}

for name, value in lakes_names.items():
    Map.addLayer(value, {}, name)


In [62]:
Map=geemap.Map()
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [30]:
Map.user_rois.getInfo()

{'type': 'FeatureCollection',
 'columns': {'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Polygon',
    'coordinates': [[[-6.37207, 36.666216],
      [-6.37207, 37.803274],
      [-2.768555, 37.803274],
      [-2.768555, 36.666216],
      [-6.37207, 36.666216]]]},
   'id': '0',
   'properties': {}}]}

# Computing NDCI

In [31]:
roi=ee.Geometry.Polygon([[-6.37207, 36.666216],
      [-6.37207, 37.803274],
      [-2.768555, 37.803274],
      [-2.768555, 36.666216],
      [-6.37207, 36.666216]])

In [35]:

def maskS2clouds(image):
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
    .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    # Apply the mask and scale the image.
    masked_image = image.updateMask(mask).divide(10000)
    
    # Copy properties from the original image to the masked image.
    return masked_image.copyProperties(image, image.propertyNames())

dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
.filterDate('2022-01-01', '2024-10-30') \
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',40)) \
.filterBounds(roi)\
.map(maskS2clouds)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

Map.centerObject(roi,zoom=8)
Map.addLayer(dataset.mean().clip(roi), visualization, 'RGB')


In [36]:
dataset

In [103]:
def func_kzc(img):
    bands = img.select('B.*')
    mndwi = bands.normalizedDifference(['B3','B11']).rename('mndwi')
    thr = mndwi.gt(0.1)
    ndci = bands.normalizedDifference(['B5','B4']).rename('ndci')
    return ndci.updateMask(thr) \
    .copyProperties(img,['system:time_start','system:time_end']) 




In [90]:
import datetime

def date_to_standard(dates):
    DT = []
    for date in dates:
        # Convert Unix timestamp to datetime object
        standard_date = datetime.datetime.utcfromtimestamp(date / 1000)
        # Append the formatted date to the list
        DT.append(standard_date.strftime('%Y-%m-%d %H:%M:%S'))
    return DT

In [119]:
def timplapse_NDCI(geometry):
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterDate('2022-01-01', '2024-10-01') \
        .filterBounds(geometry) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        
    visualization = {
        'min': 0.0,
        'max': 0.3,
        'bands': ['B4', 'B3', 'B2'],}

    m=geemap.Map()
    m.addLayer(sentinel.mean().clip(geometry), visualization, 'RGB')

    ndci=sentinel.map(func_kzc)

    date_list = ndci.aggregate_array('system:time_start').getInfo()
    standard_dates=date_to_standard(date_list)

    resultbands = ndci.toBands()
    visparams = {
        'palette': [
            '#0000FF',  # Blue
            '#1C78D4',  # Bright Navy Blue
            '#EAF12D',  # Dandelion (Yellow)
            '#F1D809',  # Safety Yellow
            '#82B328',  # Dark Lemon Lime
            '#46892B',  # Sap Green
            '#00FF00'   # Green
        ],
        'stretch': 'percentile', 
        'percentiles': [2, 95]  
    }

    
    # Add the result to the map.
    m.add_time_slider(resultbands, visparams,labels=standard_dates,
                        time_interval=4 )
    m.centerObject(geometry, 14)
    m.add_colorbar(
        visparams, label="NDCI", 
        layer_name="NDCI", orientation="vertical")
    return m

In [120]:
timplapse_NDCI(canales_roi)

Map(center=[37.15573139536438, -3.458633499992257], controls=(WidgetControl(options=['position', 'transparent_…

In [121]:
timplapse_NDCI(canales_roi)

Map(center=[37.15573139536438, -3.458633499992257], controls=(WidgetControl(options=['position', 'transparent_…

In [122]:
timplapse_NDCI(quentar_roi)

Map(center=[37.20589588663589, -3.4329915000138502], controls=(WidgetControl(options=['position', 'transparent…

In [123]:
timplapse_NDCI(colomera_roi)

Map(center=[37.41165370628457, -3.7115334999992284], controls=(WidgetControl(options=['position', 'transparent…

In [124]:
timplapse_NDCI(iznajar_roi)

Map(center=[37.270629543023176, -4.30200550000238], controls=(WidgetControl(options=['position', 'transparent_…

In [125]:
timplapse_NDCI(bermejales_roi)

Map(center=[36.97815910180854, -3.88512599999974], controls=(WidgetControl(options=['position', 'transparent_b…

In [126]:
timplapse_NDCI(cubillas_roi)

Map(center=[37.27530847595103, -3.673381999989766], controls=(WidgetControl(options=['position', 'transparent_…

In [127]:
timplapse_NDCI(elgergal_roi)

Map(center=[37.58425514592419, -6.04505549999495], controls=(WidgetControl(options=['position', 'transparent_b…

In [48]:
'''
for name, geometry in lakes_names.items():
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterDate('2023-01-01', '2024-10-01') \
        .filterBounds(geometry) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 45)) \
        .map(maskS2clouds) \
        .map(func_kzc)

    date_list = sentinel.aggregate_array('system:time_start').getInfo()
    standard_dates=date_to_standard(date_list)

    resultbands = sentinel.toBands()
    visparams = {
        'palette': [
            '#0000FF',  # Blue
            '#0044FF',
            '#0088FF',
            '#00CCFF',
            '#00FFFF',  # Cyan
            '#00FFCC',
            '#00FF88',
            '#00FF44',
            '#00FF00'   # Green
        ]
    }
    
    # Add the result to the map.
    Map.add_time_slider(resultbands, visparams,labels=standard_dates,
                        time_interval=3, layer_name=f'NDCI Result for {name}' )
    Map

# Display the map.
'''